In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
import re

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [25]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

In [35]:
environment_model = ChatOpenAI(model="ft:gpt-3.5-turbo-1106:yield-inc::8TJSa7YA")
#environment_model = ChatOpenAI(model="gpt-4-0613")
environment_model.temperature = 0.8

### Get Tasks

In [3]:
interactions = []
with open('dbbench-interactions-syn_env_syn_chat-raw-5.jsonl', 'r') as file:
    for line in file:
        interactions.append(json.loads(line))



In [8]:
interactions[-1][2]

{'from': 'human',
 'value': 'Which hospital has the highest number of patients in the Intensive Care Unit (ICU)?. There are 2 tables involved with this task. The name of the first table is HospitalInfo, with headers: HospitalID, HospitalName, HospitalAddress, HospitalCity, HospitalState, HospitalZipcode. The second table is PatientInfo, with headers: PatientID, HospitalID (refers to HospitalInfo), PatientName, PatientDOB, PatientAdmissionDate, PatientDischargeDate, PatientCondition, PatientRoomType (ICU, General, etc.).'}

In [10]:
tasks = [x[2]['value'] for x in interactions]

In [11]:
tasks[0:3]

["Query to find the total sales made by the employee 'John Doe'.. There are 2 tables involved in this task. The first table is named 'employees' with headers 'emp_id', 'first_name', 'last_name', 'email', 'phone_number'. The second table is named 'sales' with headers 'sale_id', 'emp_id', 'product_id', 'quantity', 'sale_date', 'total_price'. ",
 "Show the list of books that are currently issued and not returned.. The library database has two tables. The first table is 'books' with headers 'book_id', 'title', 'author', 'genre', 'publish_date'. The second table is 'issue_records' with headers 'record_id', 'book_id', 'member_id', 'issue_date', 'return_date'. ",
 "Find the average age of patients diagnosed with 'Influenza'.. The hospital database has two tables. The first table is named 'patients' with headers 'patient_id', 'first_name', 'last_name', 'birth_date', 'gender', 'contact'. The second table is named 'diagnoses' with headers 'diagnosis_id', 'patient_id', 'disease', 'diagnosis_date'

### Create Data

In [21]:
environment_prompt_template = '''Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below

Initial Database state:
{}

First command:
{}

The user is working on the following task. The Database may include state that helps the user complete the task:
{}

Please only respond in rawMySQL format (with no extra formatting or commentary) for a user of  mysql-connector-python, for example, if the result is 59.555, the result would be presented as [('59.555',)]. After responding, end your response.
'''

mysql_agent_prompt = '''I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining thoroughly, every round you can choose to operate or to answer. your operation should be like this: Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output. If you are done operating, and you want to commit your final answer, then write down: Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer. Your answer should be accurate. Your answer must be exactly the same as the correct answer. If the question is about modifying the database, then after done operation, your answer field can be anything. If your response cannot match any pattern I mentioned earlier, you will be judged as FAIL immediately. Your input will be raw MySQL response, you have to deal with it by yourself.'''

In [31]:
mysql_agent_prompt_improved = '''I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. 
You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining 
thoroughly, every round you can choose to operate or to answer. your operation should be like this: 
Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format 
without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. 
I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you 
and give you the output. If the output is zero, or empty, you should always double check that you haven't made a mistake before submitting a final answer.
You can double check by removing limitations from your previous SQL statement until you get non-zero or non-empty results.
If you are done operating, and you want to commit your final answer, then write down: 
Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your 
answer. You must ALWAYS provide SQL or a Final Answer. I expect an accurate and correct answer. Your answer should be accurate. 
Your answer must be exactly the same as the correct answer. If the question is about modifying the database, then after you are done with operation, 
your answer field can be anything. If your response cannot match any pattern I mentioned earlier, 
you will be judged as FAIL immediately. Your input will be raw MySQL response, you have to deal with it by yourself.'''

In [22]:
def process(data):
    return (data.split("..")[0], '\n'.join(data.split("..")[1:]))

In [23]:
process(tasks[0])

("Query to find the total sales made by the employee 'John Doe'",
 " There are 2 tables involved in this task. The first table is named 'employees' with headers 'emp_id', 'first_name', 'last_name', 'email', 'phone_number'. The second table is named 'sales' with headers 'sale_id', 'emp_id', 'product_id', 'quantity', 'sale_date', 'total_price'. ")

In [36]:
num_turns = 20

all_agents = []
all_environments = []
start = 0

for x in range(start,start+len(tasks)): 
    task = tasks[x]
    task_, environment_info = process(task)
    print("Task {}:".format(x), task_)
    agent_messages = [
        HumanMessage(content=mysql_agent_prompt_improved),
        AIMessage(content="Ok."),
        HumanMessage(content=task)
    ]
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
    if first_sql_block:
        sql_code = first_sql_block.group(1).strip()
    else:
        sql_code = ""
    environment_prompt = environment_prompt_template.format(environment_info, sql_code, task_)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    environment_result = environment_model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
        if first_sql_block:
            sql_code = first_sql_block.group(1).strip()
        else:
            sql_code = ""
        environment_messages.append(HumanMessage(content=sql_code))
        agent_messages.append(agent_response)
        if "Final Answer:" in agent_response.content:
            break
        environment_result = environment_model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))

Task 0: Query to find the total sales made by the employee 'John Doe'
To find the total sales made by 'John Doe', we need to join the 'employees' table and the 'sales' table using the 'emp_id' field. This will allow us to find the sales entries associated with the given employee.

First, we have to find out the 'emp_id' of 'John Doe'. Let's find his 'emp_id' in the 'employees' table.

Action: Operation 
```sql
SELECT emp_id FROM employees WHERE first_name='John' AND last_name='Doe';
```
[(1,)]
John Doe's employee ID (emp_id) is 1. 

Now, we will use this emp_id to find the total sales made by John Doe. We will sum the 'total_price' from the 'sales' table where 'emp_id' is 1. 

Action: Operation 
```sql
SELECT SUM(total_price) FROM sales WHERE emp_id=1;
```
[(24900,)]
Action: Answer
Final Answer: [24900]
Task 1: Show the list of books that are currently issued and not returned
To solve your problem, we need to find the books that are currently issued and not returned yet. We have two ta

KeyboardInterrupt: 

In [37]:
    all_agents[0]

[HumanMessage(content='I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. \nYou have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining \nthoroughly, every round you can choose to operate or to answer. your operation should be like this: \nAction: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format \nwithout any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. \nI will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you \nand give you the output. If the output is zero, or empty, you should always double check that you haven\'t made a mistake before submitting a final answer.\nYou can double check by removing limitations from your previous SQL statement until you get non-zero or non-empty results.\nIf you are done operating, an

In [42]:
all_agents[0][1].type

'ai'

In [43]:
all_environments[0]

[HumanMessage(content="Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\n There are 2 tables involved in this task. The first table is named 'employees' with headers 'emp_id', 'first_name', 'last_name', 'email', 'phone_number'. The second table is named 'sales' with headers 'sale_id', 'emp_id', 'product_id', 'quantity', 'sale_date', 'total_price'. \n\nFirst command:\nSELECT emp_id FROM employees WHERE first_name='John' AND last_name='Doe';\n\nThe user is working on the following task. The Database may include state that helps the user comple